# Proyecto Quimioterapia

*MA4703-1 Control Óptimo: Teoría y Laboratorio* \
**Integrantes:** Gisela Abarca A. - Natalia González R. \
**Profesor:** Héctor Ramírez \
**Auxiliares:** Sebastián Pincheira - Joaquín Márquez \
**Ayudante:** Fraick Reyes 

In [1]:
# Librerías
using DifferentialEquations
using Plots
using LinearAlgebra
using Printf
using OptimalControl
using ForwardDiff
using NLPModelsIpopt
using Pkg
using Statistics
using Ipopt
using Optim
using DataFrames
using CSV

[ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
[ Info: Precompiling DiffEqBase [2b5f629d-d688-5b77-993f-72d75c75574e] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing DiffEqBase [2b5f629d-d688-5b77-993f-72d75c75574e].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
[ Info: Precompiling DiffEqNoiseProcess [77a26b50-5914-5dd7-bc55-306e6241c503] 
┌ Warnin

## Sin quimioterapia

In [2]:
# Crear carpeta
results_dir = "resultados_quimioterapia"
if !isdir(results_dir)
    mkdir(results_dir)
end

# Parámetros del modelo (valores del paper Kirschner et al. 1997)
params = (s = 10.0, μ_T = 0.02, r = 0.03, T_max = 1500.0,
          k₁ = 2.4e-5, k₂ = 3.0e-3, μ_b = 0.24, N = 1200.0, μ_V = 2.4)

# Sistema sin tratamiento (u = 0)
function sistema_sin_control!(du, u, p, t)
    T, T_ast, T_astast, V = u
    s, μ_T, r, T_max, k₁, k₂, μ_b, N, μ_V = p
    
    total_cells = T + T_ast + T_astast
    du[1] = s/(1+V) - μ_T*T + r*T*(1 - total_cells/T_max) - k₁*V*T
    du[2] = k₁*V*T - μ_T*T_ast - k₂*T_ast
    du[3] = k₂*T_ast - μ_b*T_astast
    du[4] = N*μ_b*T_astast - k₁*V*T - μ_V*V
end

# Condiciones iniciales (infección temprana)
u0 = [1000.0, 0.0, 0.0, 1e-3]  # T, T*, T**, V
tspan = (0.0, 2000.0)  # 2000 días

# Resolver el sistema sin control
prob_sin_control = ODEProblem(sistema_sin_control!, u0, tspan, params)
sol_sin_control = solve(prob_sin_control, Tsit5(), saveat=1.0)

# Evolución temporal completa
println("Generando simulación 1: Evolución temporal completa...")

p1 = plot(sol_sin_control, idxs=1, label="T (células sanas)", linewidth=3, 
          title="A. Células T Sanas", xlabel="Tiempo (días)", ylabel="Células/mm³",
          legend=:right, grid=true, color=:blue)

p2 = plot(sol_sin_control, idxs=2, label="T* (latentes)", linewidth=2,
          title="B. Células Infectadas Latentes", xlabel="Tiempo (días)", 
          ylabel="Células/mm³", legend=:topright, grid=true, color=:orange)

p3 = plot(sol_sin_control, idxs=3, label="T** (activas)", linewidth=2,
          title="C. Células Infectadas Activas", xlabel="Tiempo (días)",
          ylabel="Células/mm³", legend=:topright, grid=true, color=:red)

p4 = plot(sol_sin_control, idxs=4, label="V (virus)", linewidth=2, 
          title="D. Carga Viral", xlabel="Tiempo (días)", ylabel="Virus/mm³",
          legend=:topright, grid=true, color=:purple, yscale=:log10)

plot_completo = plot(p1, p2, p3, p4, layout=(2,2), size=(1200, 800))
savefig(plot_completo, joinpath(results_dir, "01_evolucion_completa.png"))

# Comparación T vs V en escala semi-log
println("Generando simulación 2: Comparación T vs V...")

p_T_V = plot(sol_sin_control.t, sol_sin_control[1,:], label="Células T sanas", 
             linewidth=3, color=:blue, xlabel="Tiempo (días)", 
             ylabel="Células T (escala lineal)", legend=:left, grid=true)
p_V = plot(sol_sin_control.t, sol_sin_control[4,:], label="Carga viral", 
           linewidth=3, color=:red, xlabel="Tiempo (días)", 
           ylabel="Virus (escala log)", yscale=:log10, legend=:right, grid=true)

plot_comparacion = plot(p_T_V, p_V, layout=(2,1), size=(1000, 600),
                        title="Comparación: Células T vs Carga Viral")
savefig(plot_comparacion, joinpath(results_dir, "02_comparacion_T_vs_V.png"))

# Análisis de sensibilidad - Variar parámetro N
println("Generando simulación 3: Sensibilidad al parámetro N...")

N_values = [800, 1000, 1200, 1400]  # Diferentes valores de N
colors = [:red, :orange, :green, :blue]
plots_N = []

for (i, N_val) in enumerate(N_values)
    params_mod = (s=10.0, μ_T=0.02, r=0.03, T_max=1500.0,
                 k₁=2.4e-5, k₂=3.0e-3, μ_b=0.24, N=N_val, μ_V=2.4)
    
    prob_mod = ODEProblem(sistema_sin_control!, u0, tspan, params_mod)
    sol_mod = solve(prob_mod, Tsit5(), saveat=1.0)
    
    p = plot(sol_mod.t, sol_mod[1,:], label="N = $N_val", 
             linewidth=2, color=colors[i], grid=true,
             xlabel="Tiempo (días)", ylabel="Células T sanas/mm³")
    push!(plots_N, p)
end

plot_sensibilidad_N = plot(plots_N..., layout=(2,2), size=(1000, 600),
                           title="Sensibilidad al Parámetro N (Producción viral)")
savefig(plot_sensibilidad_N, joinpath(results_dir, "03_sensibilidad_N.png"))

# Análisis de sensibilidad - Variar tasa de infección k₁
println("Generando simulación 4: Sensibilidad al parámetro k₁...")

k1_values = [1.0e-5, 2.4e-5, 5.0e-5, 1.0e-4]
colors_k1 = [:green, :blue, :orange, :red]
plots_k1 = []

for (i, k1_val) in enumerate(k1_values)
    params_mod = (s=10.0, μ_T=0.02, r=0.03, T_max=1500.0,
                 k₁=k1_val, k₂=3.0e-3, μ_b=0.24, N=1200.0, μ_V=2.4)
    
    prob_mod = ODEProblem(sistema_sin_control!, u0, tspan, params_mod)
    sol_mod = solve(prob_mod, Tsit5(), saveat=1.0)
    
    p = plot(sol_mod.t, sol_mod[1,:], 
             label=@sprintf("k₁ = %.1e", k1_val), 
             linewidth=2, color=colors_k1[i], grid=true,
             xlabel="Tiempo (días)", ylabel="Células T sanas/mm³")
    push!(plots_k1, p)
end

plot_sensibilidad_k1 = plot(plots_k1..., layout=(2,2), size=(1000, 600),
                            title="Sensibilidad al Parámetro k₁ (Tasa de infección)")
savefig(plot_sensibilidad_k1, joinpath(results_dir, "04_sensibilidad_k1.png"))

# Comportamiento a largo plazo (5000 días)
println("Generando simulación 5: Comportamiento a largo plazo...")

tspan_largo = (0.0, 5000.0)
prob_largo = ODEProblem(sistema_sin_control!, u0, tspan_largo, params)
sol_largo = solve(prob_largo, Tsit5(), saveat=10.0)

p_largo_T = plot(sol_largo.t, sol_largo[1,:], label="Células T", 
                 linewidth=3, color=:blue, grid=true,
                 xlabel="Tiempo (días)", ylabel="Células/mm³",
                 title="Comportamiento a Largo Plazo - Células T")

p_largo_V = plot(sol_largo.t, sol_largo[4,:], label="Virus", 
                 linewidth=3, color=:red, grid=true, yscale=:log10,
                 xlabel="Tiempo (días)", ylabel="Virus/mm³",
                 title="Comportamiento a Largo Plazo - Carga Viral")

plot_largo = plot(p_largo_T, p_largo_V, layout=(2,1), size=(1000, 600))
savefig(plot_largo, joinpath(results_dir, "05_comportamiento_largo_plazo.png"))

# Diagrama de fases T vs V
println("Generando simulación 6: Diagrama de fases...")

# Usamos una solución con más puntos para el diagrama de fases
prob_fases = ODEProblem(sistema_sin_control!, u0, (0.0, 3000.0), params)
sol_fases = solve(prob_fases, Tsit5(), saveat=0.1)

plot_fases = plot(sol_fases[1,:], sol_fases[4,:], 
                  label="Trayectoria T-V", linewidth=2, color=:purple,
                  xlabel="Células T sanas (mm³)", ylabel="Carga viral (mm³)",
                  title="Diagrama de Fases: T vs V", grid=true,
                  yscale=:log10, legend=:topright)

# Agregar punto inicial y final
scatter!([u0[1]], [u0[4]], label="Inicio", markersize=8, color=:green)
scatter!([sol_fases[1,end]], [sol_fases[4,end]], label="Final", markersize=8, color=:red)

savefig(plot_fases, joinpath(results_dir, "06_diagrama_fases.png"))

println("Calculando R₀ y equilibrios...")

function calcular_equilibrios(p)
    s, μ_T, r, T_max, k₁, k₂, μ_b, N, μ_V = p
    
    # Equilibrio libre de infección T₀
    T₀ = (T_max/(2*r)) * (r - μ_T + sqrt((r - μ_T)^2 + 4*r*s/T_max))
    
    # Número reproductivo básico R₀
    R₀ = (N * k₁ * k₂ * T₀) / ((μ_T + k₂) * (k₁ * T₀ + μ_V) * μ_b)
    
    return T₀, R₀
end

T₀, R₀ = calcular_equilibrios(params)

println("¡Simulaciones completadas!")
println("Resultados guardados en la carpeta: $results_dir")
println("Equilibrio libre T₀ = $(round(T₀, digits=2))")
println("Número reproductivo R₀ = $(round(R₀, digits=4))")

Generando simulación 1: Evolución temporal completa...
Generando simulación 2: Comparación T vs V...
Generando simulación 3: Sensibilidad al parámetro N...
Generando simulación 4: Sensibilidad al parámetro k₁...
Generando simulación 5: Comportamiento a largo plazo...
Generando simulación 6: Diagrama de fases...
Calculando R₀ y equilibrios...
¡Simulaciones completadas!
Resultados guardados en la carpeta: resultados_quimioterapia
Equilibrio libre T₀ = 1000.0
Número reproductivo R₀ = 6.4572


## Con quimioterapia

*Sistema Dinámico Controlado*

$$
\begin{align}
    \frac{dT}{dt} &= \frac{s}{1+V} - \mu_T T + rT \left(1 - \frac{T + T^* + T^{**}}{T_{\text{max}}}\right) - k_1 V T \\
    \frac{dT^*}{dt} &= k_1 V T - \mu_T T^* - k_2 T^* \\
    \frac{dT^{**}}{dt} &= k_2 T^* - \mu_b T^{**} \\
    \frac{dV}{dt} &= (1 - u(t)) N \mu_b T^{**} - k_1 V T - \mu_V V
\end{align}
$$
*Restricciones:*
$$
\begin{align*}
    u(t) &\in [0, u_{\text{max}}], \quad 0 \leq u_{\text{max}} \leq 1 \\
    T(0) &= T_0, \quad T^*(0) = T^*_0, \quad T^{**}(0) = T^{**}_0, \quad V(0) = V_0 \\
    t &\in [0, t_f]
\end{align*}
$$

In [3]:
# Crear carpeta 
results_control_dir = "resultados_control_optimo"
if !isdir(results_control_dir)
    mkdir(results_control_dir)
end

# Parámetros del modelo (valores del paper Kirschner et al. 1997)
params = (s = 10.0, μ_T = 0.02, r = 0.03, T_max = 1500.0,
          k₁ = 2.4e-5, k₂ = 3.0e-3, μ_b = 0.24, N = 1200.0, μ_V = 2.4)

# Parámetros de peso para las funciones objetivo
B = 100.0  
Q = 1.0    
R = 10.0
P = 0.1   
S = 50.0   
M = 50.0

# Tiempo de simulación
t0 = 0.0
tf = 1500.0

# Condiciones iniciales
T0 = 800.0   
T_ast0 = 5.0     
T_astast0 = 1.0 
V0 = 10     

x0 = [T0, T_ast0, T_astast0, V0]

# Límites del control
u_min = 0.0
u_max = 1.0

1.0

### Función objetivo 1

*Objetivo 1: Maximización de Células Sanas con Costo de Toxicidad* (cambie los signos de Quimioterapia porque en el paper maximizaban con los mismos signos)

$$
\begin{align*}
    \min_{u(\cdot)} \quad & J_1(u) = \int_{0}^{t_f} \left[ -T(t) + \frac{1}{2} B u(t)^2 \right] dt \\
    \text{s.a.:} \quad & \text{Sistema dinámico anterior} \\
    & u(t) \in [0, u_{\text{max}}]
\end{align*}
$$
*Interpretación:*
- $-T(t)$: Maximizar células T sanas (al minimizar su negativo)
- $\frac{1}{2}Bu(t)^2$: Penalizar el exceso de tratamiento
- $B > 0$: Parámetro de peso que balancea los objetivos


In [ ]:
# Problema 1
@def ocp1 begin
    t ∈ [t0, tf], time
    x ∈ R⁴, state
    u ∈ R, control
    T = x₁
    T_ast = x₂
    T_astast = x₃
    V = x₄
    
    # Sistema dinámico
    s = params.s
    μ_T = params.μ_T
    r = params.r
    T_max = params.T_max
    k₁ = params.k₁
    k₂ = params.k₂
    μ_b = params.μ_b
    N = params.N
    μ_V = params.μ_V
    
    ẋ(t) == [s/(1+V(t)) - μ_T*T(t) + r*T(t)*(1 - (T(t) + T_ast(t) + T_astast(t))/T_max) - k₁*V(t)*T(t), 
        k₁*V(t)*T(t) - μ_T*T_ast(t) - k₂*T_ast(t),
        k₂*T_ast(t) - μ_b*T_astast(t), 
        (1 - u(t)) * N * μ_b * T_astast(t) - k₁*V(t)*T(t) - μ_V*V(t)  ]
    
    # Restricciones
    u_min <= u(t) <= u_max
    x(t0) == x0
    x(t) >= [0,0,0,0]

    # Función objetivo
    ∫(-T(t) + 0.5 * B * u(t)^2) → min
    
end

sol1 = solve(ocp1)

Resolviendo Problema 1: Maximización de Células Sanas...
▫ This is OptimalControl version v1.1.1 running with: direct, adnlp, ipopt.

▫ The optimal control problem is solved with CTDirect version v0.16.3.

   ┌─ The NLP is modelled with ADNLPModels and solved with NLPModelsIpopt.
   │
   ├─ Number of time steps⋅: 250
   └─ Discretisation scheme: 

### Función objetivo 2

*Objetivo 2: Control Agresivo de Carga Viral*

$$
\begin{align*}
    \min_{u(\cdot)} \quad & J_2(u) = \int_{0}^{t_f} \left[ Q V(t)^2 + R u(t)^2 - P T(t) \right] dt \\
    \text{sujeto a:} \quad & \text{Sistema dinámico anterior} \\
    & u(t) \in [0, u_{\text{max}}]
\end{align*}
$$

*Interpretación:*

- $Q V(t)^2$: Penalizar fuertemente la carga viral (término cuadrático)
- $Ru(t)^2$: Penalizar el exceso de tratamiento
- $-P T(t)$: Beneficio por células T sanas
- $Q, R, P > 0$: Parámetros de peso


In [ ]:
# Problema 2
@def ocp2 begin
    t ∈ [t0, tf], time
    x ∈ R⁴, state
    u ∈ R, control
    T = x₁
    T_ast = x₂
    T_astast = x₃
    V = x₄
    
    # Sistema dinámico (mismo que problema 1)
    s = params.s
    μ_T = params.μ_T
    r = params.r
    T_max = params.T_max
    k₁ = params.k₁
    k₂ = params.k₂
    μ_b = params.μ_b
    N = params.N
    μ_V = params.μ_V
     
     ẋ(t) == [s/(1+V(t)) - μ_T*T(t) + r*T(t)*(1 - (T(t) + T_ast(t) + T_astast(t))/T_max) - k₁*V(t)*T(t), 
        k₁*V(t)*T(t) - μ_T*T_ast(t) - k₂*T_ast(t),
        k₂*T_ast(t) - μ_b*T_astast(t), 
        (1 - u(t)) * N * μ_b * T_astast(t) - k₁*V(t)*T(t) - μ_V*V(t)  ]

    # Restricciones
    u_min <= u(t) <= u_max
    x(t0) == x0
    x(t) >= [0,0,0,0]
    
    # Función objetivo
    ∫(Q * V(t)^2 + R * u(t)^2 - P * T(t)) → min
    
end

sol2 = solve(ocp2)

### Función objetivo 3

*Objetivo 3: Minimización de Células Infectadas junto al Costo de Toxicidad* 

$$
\begin{align*}
    \min_{u(\cdot)} \quad & J_3(u) = \int_{0}^{t_f} \left[ MT^{**}(t) + QV(t) + \frac{1}{2}B u(t)^2 \right] dt \\
    \text{s.a.:} \quad & \text{Sistema dinámico anterior} \\
    & u(t) \in [0, u_{\text{max}}]
\end{align*}
$$

*Interpretación:*
- $MT^{**}(t)$: Minimizar células infectadas activas
- $QV(t)$: Reducir carga viral (objetivo secundario)
- $\frac{1}{2}B u(t)^2$: Penalizar el exceso de tratamiento

In [ ]:
# Problema 3
@def ocp3 begin
    t ∈ [t0, tf], time
    x ∈ R⁴, state
    u ∈ R, control
    T = x₁
    T_ast = x₂
    T_astast = x₃
    V = x₄
    
    # Sistema dinámico (mismo que problema 1)
    s = params.s
    μ_T = params.μ_T
    r = params.r
    T_max = params.T_max
    k₁ = params.k₁
    k₂ = params.k₂
    μ_b = params.μ_b
    N = params.N
    μ_V = params.μ_V
     
     ẋ(t) == [s/(1+V(t)) - μ_T*T(t) + r*T(t)*(1 - (T(t) + T_ast(t) + T_astast(t))/T_max) - k₁*V(t)*T(t), 
        k₁*V(t)*T(t) - μ_T*T_ast(t) - k₂*T_ast(t),
        k₂*T_ast(t) - μ_b*T_astast(t), 
        (1 - u(t)) * N * μ_b * T_astast(t) - k₁*V(t)*T(t) - μ_V*V(t)  ]

    # Restricciones
    u_min <= u(t) <= u_max
    x(t0) == x0
    x(t) >= [0,0,0,0]
    
    
    # Función objetivo
    ∫( M * T_astast(t) + Q * V(t) + 0.5 * B * u(t)^2) → min
    
end

sol3 = solve(ocp3)

### Análisis Comparativo

In [ ]:
# ANÁLISIS COMPARATIVO
println("\n=== ANÁLISIS COMPARATIVO ===")

# Calcular métricas para cada solución
function calculate_metrics(sol, label)

    t_grid = time_grid(sol)
    
    xfun = state(sol)
    ufun = control(sol)

    states = [xfun(t) for t in t_grid]
    controls = [ufun(t) for t in t_grid]
    
    T_final = states[end][1]
    V_final = states[end][4]
    T_avg = mean(x[1] for x in states)
    V_avg = mean(x[4] for x in states)
    u_avg = mean(controls)
    
    println("$label:")
    println("  - Células T finales: $(@sprintf("%.2f", T_final))")
    println("  - Carga viral final: $(@sprintf("%.6f", V_final))")
    println("  - Células T promedio: $(@sprintf("%.2f", T_avg))")
    println("  - Carga viral promedio: $(@sprintf("%.6f", V_avg))")
    println("  - Control promedio: $(@sprintf("%.3f", u_avg))")
    println()
    
    return (T_final, V_final, T_avg, V_avg, u_avg)
end

# Métricas para cada problema
metrics1 = calculate_metrics(sol1, "PROBLEMA 1: Maximización Células Sanas")
metrics2 = calculate_metrics(sol2, "PROBLEMA 2: Control Agresivo Viral")
metrics3 = calculate_metrics(sol3, "PROBLEMA 3: Minimización Células Infectadas")

In [ ]:
using Statistics

# Gráficos Comparativos

# Función para valores seguros en logaritmos
safe_value(x) = x > 0 ? x : 1e-10

# Obtener datos para sol1
t1 = time_grid(sol1)
x1_fun = state(sol1)
states1 = [x1_fun(t) for t in t1]

# Obtener datos para sol2
t2 = time_grid(sol2)
x2_fun = state(sol2)
states2 = [x2_fun(t) for t in t2]

# Obtener datos para sol3
t3 = time_grid(sol3)
x3_fun = state(sol3)
states3 = [x3_fun(t) for t in t3]

# Preparar datos de carga viral seguros para log
v1_safe = [safe_value(x[4]) for x in states1]
v2_safe = [safe_value(x[4]) for x in states2]
v3_safe = [safe_value(x[4]) for x in states3]

# Comparación de estados
p1 = plot(layout=(2,2), size=(1200,800))

# Células T sanas
plot!(p1[1], t1, [x[1] for x in states1], label="Problema 1", linewidth=2)
plot!(p1[1], t2, [x[1] for x in states2], label="Problema 2", linewidth=2)
plot!(p1[1], t3, [x[1] for x in states3], label="Problema 3", linewidth=2)
plot!(p1[1], title="Células T Sanas", xlabel="Tiempo", ylabel="Células", legend=:topright)

# Células infectadas
plot!(p1[2], t1, [x[2] for x in states1], label="Problema 1", linewidth=2)
plot!(p1[2], t2, [x[2] for x in states2], label="Problema 2", linewidth=2)
plot!(p1[2], t3, [x[2] for x in states3], label="Problema 3", linewidth=2)
plot!(p1[2], title="Células Infectadas", xlabel="Tiempo", ylabel="Células", legend=:topright)

# Células latentemente infectadas
plot!(p1[3], t1, [x[3] for x in states1], label="Problema 1", linewidth=2)
plot!(p1[3], t2, [x[3] for x in states2], label="Problema 2", linewidth=2)
plot!(p1[3], t3, [x[3] for x in states3], label="Problema 3", linewidth=2)
plot!(p1[3], title="Células Latentemente Infectadas", xlabel="Tiempo", ylabel="Células", legend=:topright)

# Carga viral
plot!(p1[4], t1, v1_safe, label="Problema 1", linewidth=2, yscale=:log10)
plot!(p1[4], t2, v2_safe, label="Problema 2", linewidth=2, yscale=:log10)
plot!(p1[4], t3, v3_safe, label="Problema 3", linewidth=2, yscale=:log10)
plot!(p1[4], title="Carga Viral (escala log)", xlabel="Tiempo", ylabel="Viral Load", legend=:topright)

savefig(p1, joinpath(results_control_dir, "estados_comparativos.png"))

# Graficos individuales

# Células T sanas
p1a = plot(size=(800,600))
plot!(p1a, t1, [x[1] for x in states1], label="Problema 1", linewidth=3)
plot!(p1a, t2, [x[1] for x in states2], label="Problema 2", linewidth=3)
plot!(p1a, t3, [x[1] for x in states3], label="Problema 3", linewidth=3)
plot!(p1a, title="Comparación de Células T Sanas", xlabel="Tiempo", ylabel="Células", legend=:topright)
savefig(p1a, joinpath(results_control_dir, "celulas_T_sanas.png"))

# Células infectadas
p1b = plot(size=(800,600))
plot!(p1b, t1, [x[2] for x in states1], label="Problema 1", linewidth=3)
plot!(p1b, t2, [x[2] for x in states2], label="Problema 2", linewidth=3)
plot!(p1b, t3, [x[2] for x in states3], label="Problema 3", linewidth=3)
plot!(p1b, title="Comparación de Células Infectadas", xlabel="Tiempo", ylabel="Células", legend=:topright)
savefig(p1b, joinpath(results_control_dir, "celulas_infectadas.png"))

# Células latentemente infectadas
p1c = plot(size=(800,600))
plot!(p1c, t1, [x[3] for x in states1], label="Problema 1", linewidth=3)
plot!(p1c, t2, [x[3] for x in states2], label="Problema 2", linewidth=3)
plot!(p1c, t3, [x[3] for x in states3], label="Problema 3", linewidth=3)
plot!(p1c, title="Comparación de Células Latentemente Infectadas", xlabel="Tiempo", ylabel="Células", legend=:topright)
savefig(p1c, joinpath(results_control_dir, "celulas_latentes.png"))

# Carga viral
p1d = plot(size=(800,600))
plot!(p1d, t1, v1_safe, label="Problema 1", linewidth=3, yscale=:log10)
plot!(p1d, t2, v2_safe, label="Problema 2", linewidth=3, yscale=:log10)
plot!(p1d, t3, v3_safe, label="Problema 3", linewidth=3, yscale=:log10)
plot!(p1d, title="Comparación de Carga Viral", xlabel="Tiempo", ylabel="Viral Load (log)", legend=:topright)
savefig(p1d, joinpath(results_control_dir, "carga_viral.png"))

# Comparación de controles
p2 = plot(size=(1000,600))

# Obtener controles
u1_fun = control(sol1)
controls1 = [u1_fun(t) for t in t1]

u2_fun = control(sol2)
controls2 = [u2_fun(t) for t in t2]

u3_fun = control(sol3)
controls3 = [u3_fun(t) for t in t3]

# Controles óptimos
plot!(p2, t1, controls1, label="Problema 1", linewidth=3)
plot!(p2, t2, controls2, label="Problema 2", linewidth=3)
plot!(p2, t3, controls3, label="Problema 3", linewidth=3)
plot!(p2, title="Comparación de Controles Óptimos", xlabel="Tiempo", ylabel="u(t)", 
      legend=:topright, ylims=(0, 1))

savefig(p2, joinpath(results_control_dir, "controles_comparativos.png"))

### Principio de Pontryaguin

In [ ]:
# 1. PARÁMETROS
p_sys = (s=10.0, μ_T=0.02, r=0.03, T_max=1500.0, k1=2.4e-5, k2=3.0e-3, 
         μ_b=0.24, N=1200.0, μ_V=2.4, B=50.0, u_max=1.0) 

t0, tf = 0.0, 100.0
dt = 0.1
times = t0:dt:tf
N_steps = length(times)

x0 = [800.0, 5, 1, 10] 

# 2. INICIALIZACIÓN
u_val = zeros(N_steps) .+ 0.5  
x_val = zeros(N_steps, 4)
p_val = zeros(N_steps, 4)

# 3. FUNCIONES DIFERENCIALES

function state_dynamics!(du, u_curr, p, t)
    T, T_ast, T_astast, V = u_curr
    
    control = p.current_u 
    
    T_tot = T + T_ast + T_astast
    
    du[1] = p.s/(1+V) - p.μ_T*T + p.r*T*(1 - T_tot/p.T_max) - p.k1*V*T
    du[2] = p.k1*V*T - p.μ_T*T_ast - p.k2*T_ast
    du[3] = p.k2*T_ast - p.μ_b*T_astast
    du[4] = (1 - control)*p.N*p.μ_b*T_astast - p.k1*V*T - p.μ_V*V
end

function costate_dynamics!(dp, p_curr, p_params, t)
    p1, p2, p3, p4 = p_curr
    
    x_now = p_params.interp_x(t) 
    u_now = p_params.interp_u(t)
    
    T, T_ast, T_astast, V = x_now
    T_tot = T + T_ast + T_astast
    
    
    logist = p_params.r*(1 - T_tot/p_params.T_max)
    
    dp[1] = 1 - (p1*(-p_params.μ_T + logist - p_params.r*T/p_params.T_max - p_params.k1*V) + 
                 p2*(p_params.k1*V) + 
                 p4*(-p_params.k1*V))
                 
    dp[2] = 0 - (p1*(-p_params.r*T/p_params.T_max) + 
                 p2*(-p_params.μ_T - p_params.k2) + 
                 p3*(p_params.k2))
                 
    dp[3] = 0 - (p1*(-p_params.r*T/p_params.T_max) + 
                 p3*(-p_params.μ_b) + 
                 p4*((1-u_now)*p_params.N*p_params.μ_b))
                 
    dp[4] = 0 - (p1*(-p_params.s/((1+V)^2) - p_params.k1*T) + 
                 p2*(p_params.k1*T) + 
                 p4*(-p_params.k1*T - p_params.μ_V))
end

max_iters = 100
tol = 1e-3

println("Iniciando Forward-Backward Sweep...")

for iter in 1:max_iters
    global u_val, x_val, p_val 
    
    interp_u_fwd(t) = begin
        idx = Int(floor((t - t0)/dt)) + 1
        idx = clamp(idx, 1, N_steps)
        return u_val[idx]
    end
    
    x_temp = zeros(N_steps, 4)
    x_temp[1, :] = x0
    
    for i in 1:(N_steps-1)
        t = times[i]
        u_now = u_val[i]
        
        h = dt
        k1_x = zeros(4); state_dynamics!(k1_x, x_temp[i,:], (current_u=u_now, s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V), t)
        k2_x = zeros(4); state_dynamics!(k2_x, x_temp[i,:] + 0.5*h*k1_x, (current_u=u_now, s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V), t + 0.5*h)
        k3_x = zeros(4); state_dynamics!(k3_x, x_temp[i,:] + 0.5*h*k2_x, (current_u=u_now, s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V), t + 0.5*h)
        k4_x = zeros(4); state_dynamics!(k4_x, x_temp[i,:] + h*k3_x, (current_u=u_now, s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V), t + h)
        
        x_temp[i+1, :] = x_temp[i, :] + (h/6.0)*(k1_x + 2*k2_x + 2*k3_x + k4_x)
    end
    x_val = x_temp

    p_temp = zeros(N_steps, 4)
    p_temp[end, :] = [0.0, 0.0, 0.0, 0.0]
    
    for i in N_steps:-1:2
        t = times[i]
       
        x_now = x_val[i, :]
        u_now = u_val[i]
        
        p_params_back = (interp_x = (tt -> x_now), interp_u = (tt -> u_now), 
                         s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V)

        h = -dt
        k1_p = zeros(4); costate_dynamics!(k1_p, p_temp[i,:], p_params_back, t)
        k2_p = zeros(4); costate_dynamics!(k2_p, p_temp[i,:] + 0.5*h*k1_p, p_params_back, t + 0.5*h)
        k3_p = zeros(4); costate_dynamics!(k3_p, p_temp[i,:] + 0.5*h*k2_p, p_params_back, t + 0.5*h)
        k4_p = zeros(4); costate_dynamics!(k4_p, p_temp[i,:] + h*k3_p, p_params_back, t + h)
        
        p_temp[i-1, :] = p_temp[i, :] + (h/6.0)*(k1_p + 2*k2_p + 2*k3_p + k4_p)
    end
    p_val = p_temp
    
    u_new = zeros(N_steps)
    for i in 1:N_steps
        p4 = p_val[i, 8-4] 
        T_astast = x_val[i, 3]
        
        val = (p4 * p_sys.N * p_sys.μ_b * T_astast) / p_sys.B
        u_new[i] = clamp(val, 0.0, p_sys.u_max)
    end
    
    error = norm(u_new - u_val)
    u_val = 0.5 * u_val + 0.5 * u_new
    
    println("Iteración $iter: Error = $error")
    
    if error < tol
        println("¡Convergencia alcanzada!")
        break
    end
end

p1 = plot(times, x_val[:,1], label="Células T", lw=2, color=:blue, ylabel="Células", title= "Evolución de Células T")
savefig("FBS_HIV_Control_CelulasT.png")
p2 = plot(times, x_val[:,4], label="Virus V", lw=2, color=:red, yscale=:log10, ylabel="Carga Viral", title = "Evolución de Carga Viral")
savefig("FBS_HIV_Control_CargaViral.png")
p3 = plot(times, u_val, label="Control u(t)", lw=2, color=:green, ylims=(-0.1, 1.1), ylabel="Dosis", xlabel="Tiempo", title = "Evolución del Control de Tratamiento")
savefig("FBS_HIV_Control_Tratamiento.png")


In [ ]:
# 1. PARÁMETROS (Nuevos pesos S, M, Q, R)
p_sys = (s=10.0, μ_T=0.02, r=0.03, T_max=1500.0, k1=2.4e-5, k2=3.0e-3, 
         μ_b=0.24, N=1200.0, μ_V=2.4, u_max=1.0,
         
         S = 1.0,    
         M = 10.0,   
         Q = 0.1,    
         R = 20.0)  

t0, tf = 0.0, 50.0
dt = 0.1
times = t0:dt:tf
N_steps = length(times)

x0 = [800.0, 0.5, 0.1, 10.0] 


u_val = zeros(N_steps) .+ 0.5 
x_val = zeros(N_steps, 4)
p_val = zeros(N_steps, 4)


function state_dynamics!(du, u_curr, p, t)
    T, T_ast, T_astast, V = u_curr
    control = p.current_u
    
    T_tot = T + T_ast + T_astast
    du[1] = p.s/(1+V) - p.μ_T*T + p.r*T*(1 - T_tot/p.T_max) - p.k1*V*T
    du[2] = p.k1*V*T - p.μ_T*T_ast - p.k2*T_ast
    du[3] = p.k2*T_ast - p.μ_b*T_astast
    du[4] = (1 - control)*p.N*p.μ_b*T_astast - p.k1*V*T - p.μ_V*V
end


function costate_dynamics!(dp, p_curr, p_params, t)
    p1, p2, p3, p4 = p_curr
    x_now = p_params.interp_x(t) 
    u_now = p_params.interp_u(t)
    T, T_ast, T_astast, V = x_now
    T_tot = T + T_ast + T_astast
    
    logist = p_params.r*(1 - T_tot/p_params.T_max)
    
    dp[1] = 0.0 - (p1*(-p_params.μ_T + logist - p_params.r*T/p_params.T_max - p_params.k1*V) + 
                   p2*(p_params.k1*V) + 
                   p4*(-p_params.k1*V))
    
    
    dp[2] = -p_params.S - (p1*(-p_params.r*T/p_params.T_max) + 
                           p2*(-p_params.μ_T - p_params.k2) + 
                           p3*(p_params.k2))
    
    
    dp[3] = -p_params.M - (p1*(-p_params.r*T/p_params.T_max) + 
                           p3*(-p_params.μ_b) + 
                           p4*((1-u_now)*p_params.N*p_params.μ_b))
    
    
    dp[4] = -p_params.Q - (p1*(-p_params.s/((1+V)^2) - p_params.k1*T) + 
                           p2*(p_params.k1*T) + 
                           p4*(-p_params.k1*T - p_params.μ_V))
end

max_iters = 100
tol = 1e-3

println("Iniciando FBS - Objetivo 3 (Minimizar Infectadas)...")

for iter in 1:max_iters
    global u_val, x_val, p_val
    
    
    x_temp = zeros(N_steps, 4); x_temp[1, :] = x0
    for i in 1:(N_steps-1)
        t = times[i]; u_now = u_val[i]; h = dt
        k1_x=zeros(4); state_dynamics!(k1_x, x_temp[i,:], (current_u=u_now, s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V), t)
        x_temp[i+1, :] = x_temp[i, :] + h*k1_x
    end
    x_val = x_temp

    
    p_temp = zeros(N_steps, 4) 
    for i in N_steps:-1:2
        t = times[i]; h = -dt
        x_now = x_val[i, :]; u_now = u_val[i]
        params_back = (interp_x = (tt -> x_now), interp_u = (tt -> u_now), s=p_sys.s, μ_T=p_sys.μ_T, r=p_sys.r, T_max=p_sys.T_max, k1=p_sys.k1, k2=p_sys.k2, μ_b=p_sys.μ_b, N=p_sys.N, μ_V=p_sys.μ_V, S=p_sys.S, M=p_sys.M, Q=p_sys.Q, R=p_sys.R)
        
        k1_p=zeros(4); costate_dynamics!(k1_p, p_temp[i,:], params_back, t)
        p_temp[i-1, :] = p_temp[i, :] + h*k1_p
    end
    p_val = p_temp
    
   
    u_new = zeros(N_steps)
    for i in 1:N_steps
        p4 = p_val[i, 4]
        T_astast = x_val[i, 3]
        
        val = (p4 * p_sys.N * p_sys.μ_b * T_astast) / (2 * p_sys.R)
        u_new[i] = clamp(val, 0.0, p_sys.u_max)
    end
    
    error = norm(u_new - u_val)
    u_val = 0.5 * u_val + 0.5 * u_new
    println("Iter $iter: Error = $error")
    if error < tol break end
end


l = @layout [a; b; c]
p1 = plot(times, [x_val[:,2] x_val[:,3]], label=["T* (Latentes)" "T** (Activas)"], lw=2, ylabel="Células Infectadas")
p2 = plot(times, x_val[:,4], label="V (Virus)", color=:red, yscale=:log10, ylabel="Carga Viral")
p3 = plot(times, u_val, label="u(t) - Objetivo 3", color=:blue, lw=2, ylims=(-0.1, 1.1), ylabel="Dosis", xlabel="Días")

plot(p1, p2, p3, layout=l, size=(700, 800), title="Objetivo 3: Control de Fuente (T*, T**)")

### Hamilton-Jacobi-Bellman

Para este caso tenemos la condición de que para cada función objetivo $\lambda = 0$, por lo tatno nuestras ecuaciones de HJB para cada caso estarán dadas por:

#### Objetivo 1

Para este caso nuestro Hamiltoniano estaría definido como:
$$
\begin{align*}
\mathcal{H}(x, u, \nabla \hat{V}) &= \left(-T + \frac{1}{2} B u^2 \right) + (\nabla \hat{V})^\top f(x,u)
\end{align*}
$$
Por lo tanto tendremos que nuestra ecuación de HJB estará dada por:
$$
\begin{align*}
\frac{-\partial \hat{V}}{\partial t} &= \min_{u \in [0, u_{max}]} \mathcal{H}(x, u, \nabla \hat{V}) \\
&=\min_{u \in [0, u_{max}]} \left\{\left(-T + \frac{1}{2} B u^2 \right) + \left( \frac{\partial \hat{V}}{\partial T}\dot{T} + \frac{\partial \hat{V}}{\partial T^\ast}\dot{T}^\ast + \frac{\partial \hat{V}}{\partial T^{\ast\ast}}\dot{T}^{\ast\ast} + \frac{\partial \hat{V}}{\partial V}\dot{V} \right)\right\}
\end{align*}
$$
Ahora para poder encontrar el mínimo usaremos la condición de primer orden:
$$
\begin{align*}
\frac{\partial H}{\partial u} = 0
\end{align*}
$$
Notamos que los únicos términos que tienen a $u$ son $\frac{1}{2} B u^2$ y $-\frac{\partial \hat{V}}{\partial V} u N \mu_b T^{**}$. Por lo tanto, tendremos que por condición de primer orden:
$$
\begin{align*}
Bu - \frac{\partial \hat{V}}{\partial V} N \mu_b T^{**}  = 0 \implies u = \frac{\frac{\partial \hat{V}}{\partial V} N \mu_b T^{**}}{B}
\end{align*}
$$

In [ ]:
# Verificación Problema 1
λ1 = costate(sol1)
t_plot_1 = time_grid(sol1)
T_astast_val_1 = [state(sol1)(t)[3] for t in t_plot_1] 
lambda_V_val_1 = [λ1(t)[4] for t in t_plot_1]     

# Control teorico
u_teorico_1 = (params.N * params.μ_b .* T_astast_val_1 .* lambda_V_val_1) ./ B

# Control numerico
plot(t_plot_1, control(sol1).(t_plot_1), label="Control Solver")
plot!(t_plot_1, u_teorico_1, label="Fórmula HJB (Teórica)", linestyle=:dash)

#### Objetivo 2

Para este caso nuestro Hamiltoniano estaría definido como:
$$
\begin{align*}
\mathcal{H}(x, u, \nabla \hat{V}) &= \left(Q V(t)^2 + R u(t) ^2 - P T(t) \right) + (\nabla \hat{V})^\top f(x,u)
\end{align*}
$$
Por lo tanto tendremos que nuestra ecuación de HJB estará dada por:
$$
\begin{align*}
\frac{-\partial \hat{V}}{\partial t} &= \min_{u \in [0, u_{max}]} \mathcal{H}(x, u, \nabla \hat{V}) \\
&=\min_{u \in [0, u_{max}]} \left\{\left(Q V(t)^2 + R u(t)^2 - P T(t) \right) + \left( \frac{\partial \hat{V}}{\partial T}\dot{T} + \frac{\partial \hat{V}}{\partial T^\ast}\dot{T}^\ast + \frac{\partial \hat{V}}{\partial T^{\ast\ast}}\dot{T}^{\ast\ast} + \frac{\partial \hat{V}}{\partial V}\dot{V} \right)\right\}
\end{align*}
$$
Ahora para poder encontrar el mínimo usaremos la condición de primer orden:
$$
\begin{align*}
\frac{\partial H}{\partial u} = 0
\end{align*}
$$
Notamos que los únicos términos que tienen a $u$ son $R u^2$ y $-\frac{\partial \hat{V}}{\partial V} u N \mu_b T^{**}$. Por lo tanto, tendremos que por condición de primer orden:
$$
\begin{align*}
2 R u - \frac{\partial \hat{V}}{\partial V} N \mu_b T^{**}  = 0 \implies u = \frac{\frac{\partial \hat{V}}{\partial V} N \mu_b T^{**}}{2 R}
\end{align*}
$$

In [ ]:
# Verificación Problema 2
λ2 = costate(sol2) 
t_plot_2 = time_grid(sol2)
T_astast_val_2 = [state(sol2)(t)[3] for t in t_plot_2]
lambda_V_val_2 = [λ2(t)[4] for t in t_plot_2]         

# Control teorico
u_teorico_2 = (params.N * params.μ_b .* T_astast_val_2 .* lambda_V_val_2) ./ (2*R)

# Control numerico
plot(t_plot_2, control(sol2).(t_plot_2), label="Control Solver")
plot!(t_plot_2, u_teorico_2, label="Fórmula HJB (Teórica)", linestyle=:dash)

#### Objetivo 3

Para este caso nuestro Hamiltoniano estaría definido como:
$$
\begin{align*}
\mathcal{H}(x, u, \nabla \hat{V}) &= \left( MT^{**}(t) + QV(t) + \frac{1}{2}B u(t)^2  \right) + (\nabla \hat{V})^\top f(x,u)
\end{align*}
$$
Por lo tanto tendremos que nuestra ecuación de HJB estará dada por:
$$
\begin{align*}
\frac{-\partial \hat{V}}{\partial t} &= \min_{u \in [0, u_{max}]} \mathcal{H}(x, u, \nabla \hat{V}) \\
&=\min_{u \in [0, u_{max}]} \left\{\left( MT^{**}(t) + QV(t) + \frac{1}{2}B u(t)^2 \right) + \left( \frac{\partial \hat{V}}{\partial T}\dot{T} + \frac{\partial \hat{V}}{\partial T^\ast}\dot{T}^\ast + \frac{\partial \hat{V}}{\partial T^{\ast\ast}}\dot{T}^{\ast\ast} + \frac{\partial \hat{V}}{\partial V}\dot{V} \right)\right\}
\end{align*}
$$
Ahora para poder encontrar el mínimo usaremos la condición de primer orden:
$$
\begin{align*}
\frac{\partial H}{\partial u} = 0
\end{align*}
$$
Notamos que los únicos términos que tienen a $u$ son $\frac{1}{2} B u^2$ y $-\frac{\partial \hat{V}}{\partial V} u N \mu_b T^{**}$. Por lo tanto, tendremos que por condición de primer orden:
$$
\begin{align*}
Bu - \frac{\partial \hat{V}}{\partial V} N \mu_b T^{**}  = 0 \implies u = \frac{\frac{\partial \hat{V}}{\partial V} N \mu_b T^{**}}{B}
\end{align*}
$$

In [ ]:
# Verificación Problema 3
λ3 = costate(sol3)
t_plot_3 = time_grid(sol3)
T_astast_val_3 = [state(sol3)(t)[3] for t in t_plot_3] 
lambda_V_val_3 = [λ3(t)[4] for t in t_plot_3]        

# Control teorico
u_teorico_3 = (params.N * params.μ_b .* T_astast_val_3 .* lambda_V_val_3) ./ B

# Control numerico
plot(t_plot_3, control(sol3).(t_plot_3), label="Control Solver")
plot!(t_plot_3, u_teorico_3, label="Fórmula HJB (Teórica)", linestyle=:dash)